<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Train-test Split and Cross-Validation Lab

_Authors: Joseph Nelson (DC), Kiefer Katovich (SF)_

---

## Review of train/test validation methods

We've discussed overfitting, underfitting, and how to validate the "generalizeability" of your models by testing them on unseen data. 

In this lab you'll practice two related validation methods: 
1. **train/test split**
2. **k-fold cross-validation**

Train/test split and k-fold cross-validation both serve two useful purposes:
- We prevent overfitting by not using all the data, and
- We retain some remaining data to evaluate our model.

In the case of cross-validation, the model fitting and evaluation is performed multiple times on different train/test splits of the data.

Ultimately we can the training and testing validation framework to compare multiple models on the same dataset. This could be comparisons of two linear models, or of completely different models on the same data.


## Instructions

For your independent practice, fit **three different models** on the Boston housing data. For example, you could pick three different subsets of variables, one or more polynomial models, or any other model that you like. 

**Start with train/test split validation:**
* Fix a testing/training split of the data
* Train each of your models on the training data
* Evaluate each of the models on the test data
* Rank the models by how well they score on the testing data set.

**Then try K-Fold cross-validation:**
* Perform a k-fold cross validation and use the cross-validation scores to compare your models. Did this change your rankings?
* Try a few different K-splits of the data for the same models.

If you're interested, try a variety of response variables.  We start with **MEDV** (the `.target` attribute from the dataset load method).

In [1]:
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston

boston = load_boston()

In [14]:
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target
X.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48
505,0.04741,0.0,11.93,0.0,0.573,6.030,80.8,2.5050,1.0,273.0,21.0,396.90,7.88


### 1. Clean up any data problems

Load the Boston housing data.  Fix any problems, if applicable.

In [21]:
for c in df.columns.values:
    print(c, sum(df[c].isna()), sum(df[c].isnull()))

CRIM 0 0
ZN 0 0
INDUS 0 0
CHAS 0 0
NOX 0 0
RM 0 0
AGE 0 0
DIS 0 0
RAD 0 0
TAX 0 0
PTRATIO 0 0
B 0 0
LSTAT 0 0


### 2. Select 3-4 variables with your dataset to perform a 50/50 test train split on

- Use sklearn.
- Score and plot your predictions.

In [40]:
from sklearn.model_selection import train_test_split

features = ['AGE','CRIM','RAD']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.8, random_state=42)

In [28]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

lr = LinearRegression().fit(X_train, y_train)
y_pred = lr.predict(X_test)
metrics.mean_squared_error(y_pred, y_test)

68.55897396025553

### 3. Try 70/30 and 90/10
- Score and plot.  
- How do your metrics change?

In [6]:
# A:

### 4. Try K-Folds cross-validation with K between 5-10 for your regression. 

- What seems optimal? 
- How do your scores change?  
- What the variance of scores like?
- Try different folds to get a sense of how this impacts your score.

array([0.        , 0.05555556, 0.11111111, 0.16666667, 0.22222222,
       0.27777778, 0.33333333, 0.38888889, 0.44444444, 0.5       ])

In [48]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso

alphas = np.linspace(0.01, 0.5, 10)

mses = []

for a in alphas:
    ridgereg = Lasso(alpha=a)

    mses.append(np.mean(cross_val_score(ridgereg, X_train, y_train, cv=100, 
                        scoring='neg_mean_squared_error')))

In [53]:
mses

[-31.293894730646233,
 -31.818227946801148,
 -31.527477891830475,
 -31.419666497328272,
 -31.55018569191351,
 -31.654101530066296,
 -31.836069487990045,
 -32.103273460566854,
 -32.41139674469883,
 -32.760206503992706]

In [44]:
alphas[4:6]

array([0.22222222, 0.27777778])

In [55]:
ridgereg = Lasso(alpha=alphas[0], normalize=True)

ridgereg.fit(X_train, y_train)

# Predict with fitted model.
y_pred = ridgereg.predict(X_test)

print(metrics.mean_squared_error(y_pred, y_test))

list(zip(boston.feature_names, ridgereg.coef_))

25.518996223195643


[('CRIM', -0.07574970712776749),
 ('ZN', 0.004419733496652104),
 ('INDUS', 0.06726086615398488),
 ('CHAS', 4.943380038011323),
 ('NOX', -9.912838150969623),
 ('RM', 5.052818564554981),
 ('AGE', -0.02416782148424126),
 ('DIS', -0.8961156250355842),
 ('RAD', 0.040112905954081415),
 ('TAX', 0.0007366253898273002),
 ('PTRATIO', -1.0032794157151608),
 ('B', 0.011350619966219763),
 ('LSTAT', -0.3842038940875475)]

### 5. [Bonus] optimize the $R^2$ score

Can you optimize your R^2 by selecting the best features and validating the model using either train/test split or K-Folds?

Your code will need to iterate through the different combinations of predictors, cross-validate the current model parameterization, and determine which set of features performed best.

The number of K-folds is up to you.

> *Hint:* the `itertools` package is useful for combinations and permutations.


In [8]:
# A:

### 5.1 Can you explain what could be wrong with this approach?

In [9]:
# A:

### 6. [Bonus] Explore another target variable and practice `patsy` formulas

Can you find another response variable, given a combination of predictors, that can be predicted accurately through the exploration of different predictors in this dataset?

**Try out using patsy to construct your target and predictor matrices from formula strings.**

> *Tip: Check out pairplots, coefficients, and pearson scores.*

In [10]:
import patsy

# A: